## Imports

In [1]:
import sys
import os
import random
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import h5py
from h5py import File as HDF5File

import enum
print(enum.__file__)

/usr/lib/python3.8/enum.py


In [3]:
# Oh boy this might take a while
import tensorflow as tf
from tensorflow import keras

from keras.layers import Lambda, Input
from keras.layers import Dropout, Flatten, Dense
import keras.backend as K
from keras.models import Sequential, Model 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate

2024-01-14 14:45:42.011131: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-14 14:45:42.846579: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import yaml
with open("../config.yaml") as fin:
    config = yaml.safe_load(fin)
    data_dir = config["data_dir"]
    model_dir = config["model_dir"]
    output_dir = config["output_dir"]

In [5]:
task_name = "scalar1"

## Loading data

See `cnn/data.py`.

In [6]:
from cnn_data import load_data
X_train, Y_train, X_test, Y_test = load_data(task_name)

load_data from datafile /data/wifeng/photon-jet/data/h5/pi0_40-250GeV_100k.h5
load_data from datafile /data/wifeng/photon-jet/data/h5/gamma_40-250GeV_100k.h5
load_data from datafile /data/wifeng/photon-jet/data/h5/scalar1_40-250GeV_100k.h5
rng=Generator(PCG64), perm=[ 95232  82411 205215 ... 243280 200833 138019], n_train=21000, n_test=27900


In [7]:
print("Training shapes:")
print("  inputs:", X_train[0].shape, X_train[1].shape)
print("  labels:", Y_train.shape)

Training shapes:
  inputs: (21000, 4, 16, 1) (21000, 4, 128, 1)
  labels: (21000,)


## Build the model

In [8]:
visible1 = Input(shape=(4, 16, 1))
conv11 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)
print ('cov11.shape', conv11.shape)
print ('pool1.shape', pool11.shape)
print ('cov12.shape', conv12.shape)
print ('pool2.shape', pool12.shape)

cov11.shape (None, 4, 16, 32)
pool1.shape (None, 2, 8, 32)
cov12.shape (None, 2, 8, 16)
pool2.shape (None, 1, 4, 16)


2024-01-14 14:45:47.775824: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8982 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:88:00.0, compute capability: 7.5
2024-01-14 14:45:47.776781: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9650 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:89:00.0, compute capability: 7.5
2024-01-14 14:45:47.777667: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9650 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:b1:00.0, compute capability: 7.5
2024-01-14 14:45:47.778421: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 9650 MB memory:  -> device: 3, name: NVIDIA GeForce RTX

In [9]:
visible2 = Input(shape=(4, 128, 1))
conv21 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)
print ('cov21.shape', conv21.shape)
print ('poo21.shape', pool21.shape)
print ('cov22.shape', conv22.shape)
print ('poo22.shape', pool22.shape)

cov21.shape (None, 4, 128, 32)
poo21.shape (None, 2, 64, 32)
cov22.shape (None, 2, 64, 16)
poo22.shape (None, 1, 32, 16)


In [10]:
merge = concatenate([flat1, flat2])

In [17]:
# interpretation model
hidden1 = Dense(10, activation='relu')(merge)
hidden2 = Dense(10, activation='relu')(hidden1)
output = Dense(3, activation='sigmoid')(hidden2)
cnn = Model(inputs=[visible1, visible2], outputs=output)

In [12]:
# summarize layers
# print(cnn.summary())

In [13]:
cnn.inputs

[<KerasTensor: shape=(None, 4, 16, 1) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 4, 128, 1) dtype=float32 (created by layer 'input_2')>]

## Train the model

In [20]:
def train_iteration(lr, epochs):
    cnn.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )    
    return cnn.fit(
        X_train, Y_train,
        epochs=epochs, batch_size=100,
        validation_data=(X_test, Y_test),
    )

In [21]:
train_iteration(lr=1e-2, epochs=5)

Epoch 1/5
210/210 [==============================] - 5s 12ms/step - loss: 0.9028 - accuracy: 0.8537 - val_loss: 0.1860 - val_accuracy: 0.9415
Epoch 2/5
210/210 [==============================] - 2s 9ms/step - loss: 0.1668 - accuracy: 0.9433 - val_loss: 0.1819 - val_accuracy: 0.9428
Epoch 3/5
210/210 [==============================] - 2s 7ms/step - loss: 0.1488 - accuracy: 0.9531 - val_loss: 0.1536 - val_accuracy: 0.9482
Epoch 4/5
210/210 [==============================] - 2s 8ms/step - loss: 0.1409 - accuracy: 0.9543 - val_loss: 0.1215 - val_accuracy: 0.9605
Epoch 5/5
210/210 [==============================] - 2s 8ms/step - loss: 0.1278 - accuracy: 0.9581 - val_loss: 0.1182 - val_accuracy: 0.9615


In [22]:
train_iteration(lr=1e-3, epochs=5)

Epoch 1/5
210/210 [==============================] - 4s 10ms/step - loss: 0.0967 - accuracy: 0.9691 - val_loss: 0.1039 - val_accuracy: 0.9687
Epoch 2/5
210/210 [==============================] - 1s 7ms/step - loss: 0.0908 - accuracy: 0.9700 - val_loss: 0.1060 - val_accuracy: 0.9669
Epoch 3/5
210/210 [==============================] - 2s 9ms/step - loss: 0.0868 - accuracy: 0.9714 - val_loss: 0.0966 - val_accuracy: 0.9701
Epoch 4/5
210/210 [==============================] - 2s 8ms/step - loss: 0.0841 - accuracy: 0.9720 - val_loss: 0.0992 - val_accuracy: 0.9687
Epoch 5/5
210/210 [==============================] - 2s 7ms/step - loss: 0.0821 - accuracy: 0.9727 - val_loss: 0.1011 - val_accuracy: 0.9685


In [23]:
train_iteration(lr=1e-4, epochs=5)

Epoch 1/5
210/210 [==============================] - 4s 10ms/step - loss: 0.0751 - accuracy: 0.9751 - val_loss: 0.0941 - val_accuracy: 0.9709
Epoch 2/5
210/210 [==============================] - 1s 6ms/step - loss: 0.0739 - accuracy: 0.9760 - val_loss: 0.0945 - val_accuracy: 0.9708
Epoch 3/5
210/210 [==============================] - 2s 7ms/step - loss: 0.0733 - accuracy: 0.9766 - val_loss: 0.0943 - val_accuracy: 0.9708
Epoch 4/5
210/210 [==============================] - 1s 6ms/step - loss: 0.0729 - accuracy: 0.9764 - val_loss: 0.0935 - val_accuracy: 0.9710
Epoch 5/5
210/210 [==============================] - 1s 7ms/step - loss: 0.0725 - accuracy: 0.9763 - val_loss: 0.0938 - val_accuracy: 0.9713


In [24]:
# Save this model
os.makedirs(f"{model_dir}", exist_ok=True)

print(f"task name: {task_name}")
cnn.save(f"{model_dir}/{task_name}_cnn")

task name: scalar1
INFO:tensorflow:Assets written to: /data/wifeng/photon-jet/cnn_models_v3.0/scalar1_cnn/assets


INFO:tensorflow:Assets written to: /data/wifeng/photon-jet/cnn_models_v3.0/scalar1_cnn/assets


In [25]:
import json

os.makedirs(f"{output_dir}/cnn", exist_ok=True)
with open(f"{output_dir}/cnn/{task_name}_train_history.json", "w") as fout:
    json.dump({
        key: history.history[key] for key in ["loss", "val_loss", "acc", "val_acc"]
    }, fout)